# Configure LaminDB on the command line: `lndb`

```{note}

Everything here is tested during CI except for the very first time a user calls `lndb set`:
The code executed in that case merely asks for email confirmation.

```

In [ ]:
!lndb --help

In [ ]:
from lamindb import load_settings

## Sign up

### The user does not yet exist

You only need to sign up a single time! For a non-signed-up email, call
```
!lndb signup --email raspbear@gmx.de
```
which prints
```
Please *confirm* the sign-up email. After that, proceed to `lndb config`!

Generated login secret: O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR.
Email & secret persist in: /Users/falexwolf/.lndb/lndb.env.
Going forward, credentials are auto-loaded. In case of loss, you can always recover your secret via email.
```

### The user confirmed the email and hence, is signed up

In this case, an error message is raised:

In [ ]:
!lndb signup --email raspbear@gmx.de

### The user did not confim the email

```{warning}

In this case, calling `lndb signup` again does unfortunately not raise an error yet, but generates a new invalid secret!

You need to recover your secret via email password recovery.
```


## Login

In [ ]:
!lndb login --email "raspbear@gmx.de" --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"

## Configure LaminDB instance

### Configure with local storage

```{note}

After the first time sign up, the following prints an additional first line as below.

```

```
Completing user sign up. Only happens once!
Created lndb instance: mydata/mydata.lndb
```

In [ ]:
!lndb config --storage "mydata"

In [ ]:
settings = load_settings()

In [ ]:
settings

In [ ]:
settings.instance_name

In [ ]:
assert settings.cloud_storage == False
assert settings.instance_name == "mydata"
assert settings.user_secret == "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"
assert settings.user_id == "9ypQ1yrW"
assert settings.storage_dir.as_posix() == "mydata"
assert settings.cache_dir is None
assert settings.db == "sqlite:///mydata/mydata.lndb"
assert settings.user_email == "raspbear@gmx.de"

Remove the test directory:

In [ ]:
settings._sqlite_file.unlink()
settings.storage_dir.rmdir()

### Configure with cloud storage

In [ ]:
!lndb config --storage "s3://lamin0"

In [ ]:
settings = load_settings()

In [ ]:
settings

In [ ]:
settings.instance_name

In [ ]:
settings.cache_dir

In [ ]:
settings.db

In [ ]:
settings._sqlite_file

In [ ]:
settings._sqlite_file_local

In [ ]:
assert settings.cloud_storage == True
assert settings.user_secret == "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"
assert settings.user_id == "9ypQ1yrW"
assert str(settings.storage_dir) == "s3://lamin0"
assert str(settings._sqlite_file) == "s3://lamin0/lamin0.lndb"
assert settings.user_email == "raspbear@gmx.de"

Clean up the test case.

In [ ]:
settings._sqlite_file.unlink()
settings._sqlite_file_local.unlink()

## Edge cases

Log in without email and secret (why do we allow this??), loading it from lndb.env:

In [ ]:
!lndb login

In [ ]:
from lamindb._setup._settings_store import settings_file

In [ ]:
settings_file

In [ ]:
settings_file.unlink()

If we remove the settings file and then try to log in, we get the following error:

In [ ]:
!lndb login

If we add an email, but still don't have a secret, we get a different error:

In [ ]:
!lndb login --email raspbear@gmx.de

Now we're all good again! 😅

In [ ]:
!lndb login --email raspbear@gmx.de --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"